## Working with Climate DT data via Earh Data Hub: https://earthdatahub.destine.eu/

In [ ]:
%load_ext autoreload
%autoreload 2

### Use your DestinE credentials 

In [ ]:
%%capture cap
%run ../cacheb/cacheb-authentication.py

In [ ]:
from pathlib import Path
with open(Path.home() / ".netrc", "w") as fp:
    fp.write(cap.stdout)

In [ ]:
!cat ~/.netrc

### Preview the dataset

In [ ]:
import xarray as xr

url = "https://cacheb.dcms.destine.eu/d1-climate-dt/ScenarioMIP-SSP3-7.0-IFS-NEMO-0001-high-sfc-v0.zarr"

ds = xr.open_dataset(
    url, 
    chunks={}, 
    engine="zarr", 
    storage_options={"client_kwargs": {"trust_env": True}}
)
ds

⚠️ This dataset is huge!

### Narrow down the selection

In [ ]:
REGION = {"N": 55, "W": 2, "S": 30, "E":23} #Italy
DATETIME = "2022-05-22T12:00"

In [ ]:
selection = ds.t2m.sel(
    longitude = slice(REGION["W"], REGION["E"]), 
    latitude=slice(REGION["S"], REGION["N"]), 
    time=DATETIME
)
selection

In [ ]:
type(ds.t2m.data)

### Check the estimated download size before downloading the data

In [ ]:
import costing

costing.estimate_download_size(ds.t2m, selection)

In [ ]:
%%time

selection.load(); # same as selection=selection.compute()

In [ ]:
type(selection.data)

In [ ]:
selection.plot() # quick Xarray plot, not very pretty

### Comparison of ClimateDT with ERA5 Single Levels

In [ ]:
era5 = xr.open_dataset(
    "https://cacheb.dcms.destine.eu/era5/reanalysis-era5-single-levels-v0.zarr",
    storage_options={"client_kwargs":{"trust_env":True}},
    chunks={},
    engine="zarr",
)
era5_selection = era5.sel(longitude = slice(REGION["W"], REGION["E"]), latitude=slice(REGION["N"], REGION["S"]), valid_time = DATETIME).t2m
era5_selection

In [ ]:
costing.estimate_download_size(era5.t2m, era5_selection)

In [ ]:
era5_selection.load()

In [ ]:
import display
display.compare_map(selection, era5_selection, title_0= "DestinE Climate DT", title_1="ERA5 single levels")

In [ ]:
display.compare_map(selection, era5_selection, title_0= "DestinE Climate DT", title_1="ERA5 single levels", contour=True)

### One More example: daily temperature trend in Ispra, June 05, 2020-2039

In [ ]:
t2m_ispra_june_05 = ds.t2m.sel(
    latitude = 45.813, 
    longitude = 8.612,
    method = "nearest",
    time=((ds.t2m.time.dt.month == 6) & (ds.t2m.time.dt.day == 5))
)
t2m_ispra_june_05

In [ ]:
costing.estimate_download_size(ds.t2m, t2m_ispra_june_05)

In [ ]:
%%time

t2m_ispra_june_05.load();

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
%%time

sns.set_style("darkgrid")

plt.figure(figsize=(15, 6))
plt.title("Hourly Temperature on June 5 (2020–2039) in Ispra", fontsize=16)
plt.xlabel("Hour of Day", fontsize=14)
plt.ylabel("Temperature (°C)", fontsize=14)
plt.grid(True, linestyle='--', alpha=1)
plt.xticks(ticks=range(0, 24, 2))

for year, data in t2m_ispra_june_05.groupby('time.year'):
    plt.plot(range(24), data.values - 273.15, label=str(year), alpha=0.8, linewidth=2)

plt.legend(title="Year", bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0., ncol=1)
plt.show()

You can also do multiple selections and operations (typically reductions). They are all lazy untill `load()` or `compute()` or `plot()` are called:

In [ ]:
t2m_ispra_june_mean = t2m_ispra_june_05 = ds.t2m.sel(
    latitude = 45.813, 
    longitude = 8.612,
    method = "nearest",
    time=(ds.t2m.time.dt.month == 6)
).sel(time = slice('2026', '2031')).groupby('time.year').mean()
t2m_ispra_june_mean

In [ ]:
%%time

t2m_ispra_june_mean.load()

In [ ]:
t2m_ispra_june_mean.plot()